# 03. 서포트 벡터 머신
> 커널기반의 서포트 벡터머신 알고리즘을 배워보고 실습을 진행해봅시다.

- toc: true 
- badges: true
- comments: true
- categories: [Day 4]
- permalink: /svm
- exec: colab

<br><br>

### 1. 선을 조금 더 잘 그어보자

![](https://i0.wp.com/hleecaster.com/wp-content/uploads/2020/01/svm01.png?w=1372)

로지스틱 회귀 알고리즘에서 우리는 Sigmoid 함수와 Cross Entropy Loss를 활용해서 데이터를 분류하는 선(Decision Boundary)을 찾는 방법에 대해서 배웠습니다. 
<br><br>

![](https://i0.wp.com/hleecaster.com/wp-content/uploads/2020/01/svm02.png?resize=1536%2C1278)

만약 데이터가 3차원이라면 위처럼 평면(Hyperplane)으로 분리를 해내겠죠? 이러한 내용은 로지스틱 회귀와 미적분 시간에 모두 다루었습니다. 
<br><br>

![](https://i1.wp.com/hleecaster.com/wp-content/uploads/2020/01/svm03.png?w=1280)

그런데 이러한 선이나 면은 위 처럼 매우 다양하게 그릴 수 있습니다. 그렇다면 **과연 어떻게 분리해내는게 좋은 방법일까요?** 위 그림의 C나 B를 보면 파란색쪽에 너무 아슬아슬하게 붙어있습니다. **만약 값이 조금 큰 파란색 샘플이 들어오면 오답으로 예측할 것입니다.** 그에 비해 F는 어느정도는 여유가 있죠
<br><br>

![](https://i0.wp.com/hleecaster.com/wp-content/uploads/2020/01/svm04.png?w=1280)

서포트 벡터 머신은 로지스틱 회귀에서 한 단계 더 발전해서 분류하기 좋은, 최적의 선을 찾아냅니다. 분류를 위한 **선이 기본적으로 근접한 데이터 샘플들과 위치상 멀어야 한다는 것**을 전제로 하고 선을 긋는 방법에 대해 학습합니다. 
<br><br>

이렇게 가장 근접한 데이터 샘플들과의 거리를 **마진(Margin)** 이라고 합니다. 위 그림에서 파란색 영역을 의미합니다. 저렇게 **마진(Margin)** 이 넓어지면 데이터를 더욱 잘 맞출 확률이 높아집니다. 또한, 마진과 맞닿아 있는 가장 근접한 샘플들을 **서포트 벡터(Support Vector)** 라고 합니다. 이 서포트 벡터들의 위치를 이용해 마진(Margin)을 계산하고, 이 마진을 최대로 벌리도록 학습합니다. 
<br><br>

![](https://github.com/gusdnd852/bigdata-lecture/blob/master/_notebooks/img/Day4/02.png?raw=True)

서포트벡터 머신은 굉장히 수학적으로 탄탄하게 정의 된 모델입니다. 공부하기에는 상당히 어려운 난이도에 속하기 때문에 수학적인 내용은 여기에서 생략하도록 하겠습니다. 관심있으신 분들은 **'목적식과 제약식', '라그랑지안 승수법', 'KKT 조건', 'Wolf 쌍대성'** 등을 공부하시고 나서 SVM을 공부하시면 이해에 수월하실 것입니다. 만약 개인적으로 궁금하시면 저한테 물어보시면 알려드리겠습니다. 웬만한 모델은 수학적으로 다뤄드릴려고 했는데 SVM 강의하려면 최소 3일은 SVM만 해야할 것 같아서 생략합니다
<br><br>

### 2. sklearn에서 한번 돌려보자!

위에서 말한 것 처럼 SVM의 분류 경계선을 찾는 건 상당히 복잡한 문제입니다. 이걸 직접 구현하려면 밤을 새도 힘들테니 우리는 sklearn을 사용해서 구현해봅시다. 우선 sklearn.svm의 SVC를 생성합니다. 이 때 kernel은 'linear'를 선택합니다. (이 것은 조금 있다가 자세히 알아봅시다)



In [6]:
from sklearn.svm import SVC

classifier = SVC(kernel='linear')

training_points = [[1, 2], 
                   [1, 5], 
                   [2, 2], 
                   [7, 5], 
                   [9, 4], 
                   [8, 2]]

labels = [0, 0, 0, 1, 1, 1]

classifier.fit(training_points, labels) 

SVC(kernel='linear')

![](https://i0.wp.com/hleecaster.com/wp-content/uploads/2020/01/svm05.png?w=1280)

SVM을 풀면 위와 같은 그래프가 그려집니다. 여기에서 (3, 2)라는 새로운 샘플을 입력하면 당연히 눈으로만 봐도 빨간색 분류쪽(0)에 있게 됩니다. 

In [8]:
classifier.predict([[3, 2]])

array([0])

<br><br>

### 3. 파라미터 C : 이상치(Outlier)를 얼마나 허용할 것인가?

SVM은 데이터 샘플들을 올바르게 분류해내면서 마진(Margin)을 최대화 해야하는데 이상치(Outlier)를 포함한 "모든" 데이터샘플을 올바르게 분류하게 되면 마진을 키우기 어려운 경우가 있습니다. 로지스틱 회귀때도 다루었지만 이렇게 결정 구분선을 그어서 분류하는 모델들은 기본적으로 이상치(Outlier)에 굉장히 취약합니다. 그래서 SVM은 이러한 이상치에 대응하는 방법을 가지고 있습니다.
<br><br>

![](https://i1.wp.com/hleecaster.com/wp-content/uploads/2020/01/svm06.png?w=1280)

위 예제는 Hard Margin과 Soft Margin을 보여줍니다. Hard Margin의 경우 학습데이터에 있는 모든 데이터 샘플을 올바르게 분류합니다. 위의 예제에서 Hard Margin을 보면 **왼쪽 끝의 파란색 이상치와 오른쪽 끝의 빨간색 이상치** 때문에 경계선이 너무 가깝게 결정되어 마진의 크기가 매우 작아졌습니다. 이런 경우 정확도가 크게 떨어지게 됩니다. 
<br><br>

그러나 Soft Margin을 보면 이상치를 무시하고 Margin을 벌리게 됩니다. 이상치들은 맞추지 못했지만 마진을 크게 벌렸기 때문에 오히려 정확도가 좋아집니다. `sklearn`에서는 `C`라는 파라미터로 이 값을 조절할 수 있습니다. 만약 `C`값이 클 수록 Hard Margin이 되고, `C`가 작을수록 Soft Margin이 됩니다. 이 값은 돌려보면서 찾아내거나 이전에 언급한 `GridSearchCV`로 찾아낼 수 있습니다.

In [9]:
# C를 0.01로 설정
classifier = SVC(kernel='linear', C=0.01)

<br><br>

### 4. 파라미터 gamma : 어떤 커널함수를 적용할 것인가?

지금까지는 선형으로 결정 경계를 그을 수 있는 형태만 예시로 보여드렸습니다. 그런데 만약 아래와 같은 데이터셋이 있다면 어떻게 될까요?

![](https://i2.wp.com/hleecaster.com/wp-content/uploads/2020/01/svm07.png?w=1280)
<br><br>

위의 데이터셋을 보면 원형으로 결정경계선을 그려야 하는 것을 직감적으로 파악할 수 있습니다. 그런데 선형모델들로는 저런 데이터셋은 분류해낼 수가 없습니다. 따라서 여기에서는 커널함수를 사용합니다. SVM에 있어서 커널함수의 의미는 다음과 같습니다. 
<br><br>

```
차원을 상승시켜서 선형적으로 분류한 뒤, 
다시 차원을 낮추면 곡선 분류선도 만들어낼 수 있다.
```
<br>

무슨 말인지 보겠습니다. 우리는 이전에 통계시간에 정규분포(가우시안 분포)에 대해서 배웠습니다. 위와 같은 데이터를 차원을 상승 시킨뒤 정규분포 모양으로 보면 아래와 같습니다. 

![](https://github.com/gusdnd852/bigdata-lecture/blob/master/_notebooks/img/Day4/03.png?raw=True)
